In [ ]:
query_attendance = """
SELECT 
    a.student_id,
    a.year_month,
    a.hours_offered,
    a.hours_attended
FROM schema.attendance_table a
LEFT JOIN schema.enrollment_table e ON a.student_id = e.student_id
LEFT JOIN schema.institution_table i ON i.institution_id = e.institution_id
LEFT JOIN schema.state_table s ON s.state_id = i.state_id
WHERE a.{attribute_0} = %s AND s.{attribute_1} = %s
"""

In [ ]:
from db_toolkit.ssh import create_ssh_tunnel
from db_toolkit.utils import load_env, get_env_variable

load_env()

tunnel = create_ssh_tunnel(
    ssh_host=get_env_variable("SSH_HOST"),
    ssh_port=22,
    ssh_username=get_env_variable("SSH_USER"),
    ssh_private_key=get_env_variable("SSH_PRIVATE_KEY"),
    remote_bind_address=(get_env_variable("DB_HOST"), int(get_env_variable("DB_PORT")))
)

In [ ]:
from db_toolkit.threaded_queries import run_parallel_queries
from db_toolkit.utils import get_env_variable

df = run_parallel_queries(
    host="localhost",
    port=tunnel.local_bind_port,
    dbname=get_env_variable("DB_NAME"),
    user=get_env_variable("DB_USER"),
    password=get_env_variable("DB_PASSWORD"),
    query_template=query_attendance,
    target_table=('schema', 'attendance_table'),
    distinct_sources={
        'year_month': ('schema', 'attendance_table'),
        'state_abbreviation': ('schema', 'state_table')
    },
    verbose=False,
    debug=False,
    max_combinations=10  # Optional for testing
)

In [ ]:
tunnel.stop()